
# Deep Learning from a Software Engineer's Perspective

## 1. Introduction
In this notebook, we will explore the steps involved in building a deep learning model. Starting with problem identification, we'll go through data preparation, building neural networks, and finally training and evaluating a model.



## 2. Identify the Problem
**Objective:** For this project, we will classify images into categories using a neural network.



## 3. Data Preparation
- **Get the Data:** Load images, which are categorized into folders representing different classes (like cats and fish).
- **Validate Data:** Ensure that each image is correctly formatted and not corrupted.
- **Transform Data:** We resize and normalize images to make them suitable for neural networks.


In [ ]:

import os
from PIL import Image, UnidentifiedImageError

def validate_images(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            try:
                img = Image.open(os.path.join(root, file))
                img.verify()  # Check if it's a valid image
            except (IOError, UnidentifiedImageError):
                print(f"Removing corrupted image: {file}")
                os.remove(os.path.join(root, file))  # Remove the file if it's corrupted

# Define the data paths
train_data_path = "./images/train/"
val_data_path = "./images/val/"
test_data_path = "./images/test/"

# Run the validation
validate_images(train_data_path)
validate_images(val_data_path)
validate_images(test_data_path)



## 4. Neural Network Basics
- **Linear vs Nonlinear Layers:** A brief explanation of how neural networks are structured.
- **Activation Functions:** Why activation functions are important for introducing non-linearity.
- **Weights and Biases:** The parameters that neural networks learn to make accurate predictions.


In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(12288, 84)  # Input layer to hidden layer
        self.fc2 = nn.Linear(84, 50)     # Hidden layer to hidden layer
        self.fc3 = nn.Linear(50, 2)      # Output layer
        
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input
        x = F.relu(self.fc1(x))    # ReLU activation
        x = F.relu(self.fc2(x))    
        x = self.fc3(x)            # Output (no softmax for CrossEntropyLoss)
        return x

class CNNNet(nn.Module):
    def __init__(self, num_classes=2):
        super(CNNNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x



## 5. Training
- **Epoch:** Training in cycles to improve model performance.
- **Device:** Using GPU if available, otherwise using CPU.
- **Accuracy and Loss:** Monitoring model performance during training.
- **Save Model:** Save the trained model for future predictions.


In [ ]:

import torch.optim as optim

model = CNNNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
model.to(device)

def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.item()
        
        training_loss /= len(train_loader)
        
        model.eval()
        num_correct = 0
        num_examples = 0
        with torch.no_grad():
            for batch in val_loader:
                inputs, targets = batch
                inputs = inputs.to(device)
                targets = targets.to(device)
                output = model(inputs)
                loss = loss_fn(output, targets)
                valid_loss += loss.item()
                correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets).view(-1)
                num_correct += torch.sum(correct).item()
                num_examples += correct.shape[0]
        
        valid_loss /= len(val_loader)
        print(f'Epoch: {epoch + 1}/{epochs}, Training Loss: {training_loss:.2f}, Validation Loss: {valid_loss:.2f}, Accuracy: {num_correct / num_examples:.2f}')
    torch.save(model, "./model/simple_model")



## 6. Prediction
After training, we can use our model to make predictions on new images.


In [ ]:

import torch
from torchvision import transforms
from PIL import Image

data_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

model = torch.load("./model/simple_model")
labels = ['cat', 'fish']
img = Image.open("pexels-crisdip-35358-128756.jpg")
img = data_transforms(img)
img = img.unsqueeze(0)
prediction = model(img)
prediction = prediction.argmax()
print(labels[prediction])
